CAMS

- mean, median, std, min, max for NO2 values by year and year-month

- maps for both results


Yearly

In [1]:
import xarray as xr
import pandas as pd
import os
import numpy as np
from collections import defaultdict

# Define the folder containing NetCDF files
folder_path = r"E:\IPMA\CAMS\chem_multlvl\5compile_2003_2024"

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Initialize an empty list to store yearly DataFrames
df_list = []

# Process each file (each corresponding to a year)
for file in file_list:
    file_path = os.path.join(folder_path, file)
    
    ds = xr.open_dataset(file_path)  # Load without specifying chunks
    ds = ds.chunk({'time': 500})  # Rechunk after loading 

    # Extract the year from the 'time' dimension
    year = ds['time'].dt.year.values[0]  # Extract the year from the first time step
    
    # Get the total no2 data as a numpy array
    no2_data = ds['no2'].values  # Numpy array (all time steps)

    # Compute statistics using numpy's nan functions
    mean = np.nanmean(no2_data, axis=0)
    median = np.nanmedian(no2_data, axis=0)
    std = np.nanstd(no2_data, axis=0)
    max_ = np.nanmax(no2_data, axis=0)
    min_ = np.nanmin(no2_data, axis=0)
    total = np.nansum(no2_data, axis=0)
    
    # Combine results into a new dataset
    stats = xr.Dataset({
        'Mean': (['latitude', 'longitude'], mean),
        'Median': (['latitude', 'longitude'], median),
        'Std': (['latitude', 'longitude'], std),
        'Max': (['latitude', 'longitude'], max_),
        'Min': (['latitude', 'longitude'], min_),
        'Total': (['latitude', 'longitude'], total)
    }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})

    # Convert to DataFrame and restructure
    stats_df = stats.to_dataframe().reset_index()
    stats_df['Year'] = year  # Add the year directly from time
    stats_df = stats_df.set_index(['Year', 'latitude', 'longitude'])
    
    # Append to list
    df_list.append(stats_df)

# Concatenate all years into a single DataFrame
df_final = pd.concat(df_list)

# Display the DataFrame
print(df_final)


                               Mean      Median         Std           Max  \
Year latitude longitude                                                     
2003 66.0     -12.00       9.934587    6.897396  322.042561    785.751387   
              -11.25      13.503613    6.686629  128.054821    602.726658   
              -10.50      32.223992    6.429002  808.197301  43528.454563   
              -9.75        6.126002    6.376601  459.451406   3069.303563   
              -9.00       14.211845    6.182337  110.581755   1008.375983   
...                             ...         ...         ...           ...   
2024 34.5      33.00      26.441132    9.860106   39.857185    277.449799   
               33.75      22.869944   15.286166   28.458088    425.811768   
               34.50      24.225584   12.094130   35.110435    420.749023   
               35.25      64.397354   33.562416   75.269386    641.885681   
               36.00     284.576202  161.587402  308.623596   1729.437134   

In [2]:
# Convert the pandas DataFrame back to an xarray Dataset
df_final_xr = df_final.reset_index().set_index(['Year', 'latitude', 'longitude'])
df_final_xr = df_final_xr.to_xarray()

# Save the xarray Dataset to a NetCDF file
output_file_path = r"E:\IPMA\CAMS\chem_multlvl\yearly_no2_stats.nc"
df_final_xr.to_netcdf(output_file_path)

print(f"Dataset saved to: {output_file_path}")


Dataset saved to: E:\IPMA\CAMS\chem_multlvl\yearly_no2_stats.nc


In [3]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# File path
file_path = r"E:\IPMA\CAMS\chem_multlvl\yearly_no2_stats.nc"
output_folder = r"E:\IPMA\CAMS\chem_multlvl\maps_year\no2"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Concentration (µg/m³)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'Blues'

# Projection (assume data is in PlateCarree, common for lat/lon grids)
data_crs = ccrs.PlateCarree()

# Loop through years
for year in ds['Year'].values:
    fig, ax = plt.subplots(figsize=(8, 6),
                           subplot_kw={'projection': ccrs.PlateCarree()})

    # Select data for current year
    data = ds[var].sel(Year=year)

    # Plot the data
    im = ax.pcolormesh(ds['longitude'], ds['latitude'], data,
                       cmap=cmap, vmin=vmin, vmax=vmax,
                       shading='auto', transform=data_crs)

    # Add features (land, ocean, borders)
    ax.coastlines(resolution='10m')
    ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.add_feature(cfeature.LAND, facecolor='lightgray')
    ax.add_feature(cfeature.OCEAN, facecolor='lightblue')
    ax.set_extent([-12, 36, 34, 66], crs=data_crs)  # Adjust to the working area

    # Gridlines (bottom and left only)
    gl = ax.gridlines(draw_labels=True, linewidth=0)
    gl.top_labels = False
    gl.right_labels = False
    gl.bottom_labels = True
    gl.left_labels = True

    # Title and colorbar
    ax.set_title(f'{title} - {year}', fontsize=12)

    # Create a colorbar with more space
    cbar = fig.colorbar(im, ax=ax, orientation='horizontal', label='µg/m³', shrink=0.7, pad=0.1)
    
    # Adjust the colorbar's position
    cbar.ax.yaxis.set_ticks_position('right')  # Put colorbar ticks on the right side

    # Save the plot
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, f"mean_concentration_no2_{year}.png"), dpi=150)
    plt.close(fig)

print("✅ Mean temperature maps with adjusted colorbar spacing generated!")


✅ Mean temperature maps with adjusted colorbar spacing generated!


Monthly

In [4]:
import xarray as xr
import pandas as pd
import os
import numpy as np

# Define the folder containing NetCDF files
folder_path = r"E:\IPMA\CAMS\chem_multlvl\4conversion_2003_2024"

# Get a sorted list of all NetCDF files in the folder
file_list = sorted([f for f in os.listdir(folder_path) if f.endswith(".nc")])

# Initialize an empty list to store monthly DataFrames
df_list = []

# Process each file (each corresponding to a year)
for file in file_list:
    file_path = os.path.join(folder_path, file)
    
    ds = xr.open_dataset(file_path)  # Load without specifying chunks
    ds = ds.chunk({'time': 500})  # Rechunk after loading 
    
    # Extract year and month from the 'time' dimension
    ds.coords['year'] = ds['time'].dt.year
    ds.coords['month'] = ds['time'].dt.month
    
    # Get the total no2 data as a numpy array (time, latitude, longitude)
    no2_data = ds['no2'].values
    
    # Calculate statistics using numpy's nan functions along the 'time' dimension
    mean = np.nanmean(no2_data, axis=0)
    median = np.nanmedian(no2_data, axis=0)
    std = np.nanstd(no2_data, axis=0)
    max_ = np.nanmax(no2_data, axis=0)
    min_ = np.nanmin(no2_data, axis=0)
    total = np.nansum(no2_data, axis=0)
    
    # Create a new xarray Dataset for the statistics
    stats = xr.Dataset({
        'Mean': (['latitude', 'longitude'], mean),
        'Median': (['latitude', 'longitude'], median),
        'Std': (['latitude', 'longitude'], std),
        'Max': (['latitude', 'longitude'], max_),
        'Min': (['latitude', 'longitude'], min_),
        'Total': (['latitude', 'longitude'], total)
    }, coords={'latitude': ds['latitude'], 'longitude': ds['longitude']})
    
    # Convert to DataFrame and restructure
    stats_df = stats.to_dataframe().reset_index()
    stats_df['Year'] = ds['year'].values[0]  # Add the year directly from time
    stats_df['Month'] = ds['month'].values[0]  # Add the month directly from time
    stats_df = stats_df.set_index(['Year', 'Month', 'latitude', 'longitude'])
    
    # Append to list
    df_list.append(stats_df)

# Concatenate all months into a single DataFrame
df_final = pd.concat(df_list)

# Display the DataFrame
print(df_final)
df_final


                                     Mean      Median         Std  \
Year Month latitude longitude                                       
2003 1     66.0     -12.00      -8.219405    3.962533  232.057805   
                    -11.25       8.305708    4.938766   22.672953   
                    -10.50       6.307520    4.865815   74.949066   
                    -9.75        9.279520    5.240869  129.238811   
                    -9.00        4.914408    5.123740   43.354542   
...                                   ...         ...         ...   
2024 11    34.5      33.00      47.775654   32.259953   45.561016   
                     33.75      20.632299   17.653496   12.008840   
                     34.50      24.740719   18.072453   20.408901   
                     35.25      70.500656   56.087349   52.170185   
                     36.00     390.658173  400.963989  256.524567   

                                       Max          Min         Total  
Year Month latitude longitude 

Mean      Median         Std  \
Year Month latitude longitude                                       
2003 1     66.0     -12.00      -8.219405    3.962533  232.057805   
                    -11.25       8.305708    4.938766   22.672953   
                    -10.50       6.307520    4.865815   74.949066   
                    -9.75        9.279520    5.240869  129.238811   
                    -9.00        4.914408    5.123740   43.354542   
...                                   ...         ...         ...   
2024 11    34.5      33.00      47.775654   32.259953   45.561016   
                     33.75      20.632299   17.653496   12.008840   
                     34.50      24.740719   18.072453   20.408901   
                     35.25      70.500656   56.087349   52.170185   
                     36.00     390.658173  400.963989  256.524567   

                                       Max          Min         Total  
Year Month latitude longitude                                          
2003 1     66.0     -12.00      160.735413 -3624.419849  -2038.412370  
                    -11.25      226.016695  -109.627427   2059.815582  
                    -10.50      466.536344 -1034.762433   1564.265047  
                    -9.75      1444.418908 -1316.172866   2301.320967  
                    -9.00       370.857602  -289.478047   1218.773200  
...                                    ...          ...           ...  
2024 11    34.5      33.00      216.545502     3.459122  11466.157227  
                     33.75      103.198219     6.685246   4951.751953  
                     34.50      129.418121     4.878763   5937.772461  
                     35.25      294.198364     7.590793  16920.158203  
                     36.00      950.479858    32.447453  93757.960938  

[735085 rows x 6 columns]

In [5]:
# Convert the pandas DataFrame back to an xarray Dataset
df_final_xr = df_final.reset_index().set_index(['Year', 'Month', 'latitude', 'longitude'])
df_final_xr = df_final_xr.to_xarray()

# Save the xarray Dataset to a NetCDF file
output_file_path = r"E:\IPMA\CAMS\chem_multlvl\monthly_no2_stats.nc"
df_final_xr.to_netcdf(output_file_path)

print(f"Dataset saved to: {output_file_path}")

df_final_xr


Dataset saved to: E:\IPMA\CAMS\chem_multlvl\monthly_no2_stats.nc


<xarray.Dataset> Size: 35MB
Dimensions:    (Year: 22, Month: 12, latitude: 43, longitude: 65)
Coordinates:
  * Year       (Year) int64 176B 2003 2004 2005 2006 ... 2021 2022 2023 2024
  * Month      (Month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
  * latitude   (latitude) float64 344B 34.5 35.25 36.0 36.75 ... 64.5 65.25 66.0
  * longitude  (longitude) float64 520B -12.0 -11.25 -10.5 ... 34.5 35.25 36.0
Data variables:
    Mean       (Year, Month, latitude, longitude) float64 6MB 30.05 ... nan
    Median     (Year, Month, latitude, longitude) float64 6MB 13.77 ... nan
    Std        (Year, Month, latitude, longitude) float64 6MB 53.98 ... nan
    Max        (Year, Month, latitude, longitude) float64 6MB 435.6 ... nan
    Min        (Year, Month, latitude, longitude) float64 6MB 5.514 ... nan
    Total      (Year, Month, latitude, longitude) float64 6MB 7.454e+03 ... nan

In [6]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# File path
file_path = r"E:\IPMA\CAMS\chem_multlvl\monthly_no2_stats.nc"
output_folder = r"E:\IPMA\CAMS\chem_multlvl\maps_month\no2"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Concentration (µg/m³)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'Blues'

# Coordinates
lat = ds['latitude']
lon = ds['longitude']

# Loop through all years and months
for year in ds['Year'].values:
    for month in ds['Month'].values:
        data = ds[var].sel(Year=year, Month=month)

        # Skip empty or all-NaN values
        if np.isnan(data).all():
            continue

        # Plotting
        fig = plt.figure(figsize=(7, 6))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.set_extent([-12, 36, 34, 66], crs=ccrs.PlateCarree())

        im = ax.pcolormesh(lon, lat, data, cmap=cmap, vmin=vmin, vmax=vmax,
                           shading='auto', transform=ccrs.PlateCarree())

        ax.set_title(f'{title} - {year} / {month:02d}', fontsize=12)
        ax.coastlines(resolution='10m', linewidth=0.8)
        ax.add_feature(cfeature.BORDERS, linewidth=0.5)
        ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='none')

        # Gridlines (bottom and left only)
        gl = ax.gridlines(draw_labels=True, linewidth=0)
        gl.top_labels = False
        gl.right_labels = False
        gl.bottom_labels = True
        gl.left_labels = True

        # Colorbar
        cbar = plt.colorbar(im, ax=ax, orientation='horizontal', shrink=0.7, pad=0.1)
        cbar.set_label('µg/m³')
        cbar.ax.tick_params(labelsize=8)

        # Save figure
        plt.tight_layout()
        filename = f"mean_concentration_no2_{year}{month:02d}.png"
        plt.savefig(os.path.join(output_folder, filename), dpi=150)
        plt.close(fig)

print("✅ Monthly temperature maps generated for all years and months!")


✅ Monthly temperature maps generated for all years and months!


In [7]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# File path
file_path = r"E:\IPMA\CAMS\chem_multlvl\monthly_no2_stats.nc"
output_folder = r"E:\IPMA\CAMS\chem_multlvl\maps_month\no2"
os.makedirs(output_folder, exist_ok=True)

# Load dataset
ds = xr.open_dataset(file_path)

# Variable to plot
var = 'Mean'
title = 'Mean Concentration (µg/m³)'

# Global color scale limits
vmin = ds[var].min().item()
vmax = ds[var].max().item()

# Colormap
cmap = 'Blues'

# Coordinates
lat = ds['latitude']
lon = ds['longitude']

# Loop through all years
for year in ds['Year'].values:
    fig, axs = plt.subplots(3, 4, figsize=(15, 10),
                            subplot_kw={'projection': ccrs.PlateCarree()})
    fig.suptitle(f'{title} - {year}', fontsize=16)

    for i, month in enumerate(ds['Month'].values):
        row, col = divmod(i, 4)
        ax = axs[row, col]

        # Extract data
        data = ds[var].sel(Year=year, Month=month)

        # Skip if no data
        if np.isnan(data).all():
            ax.set_visible(False)
            continue

        ax.set_extent([-12, 36, 34, 66], crs=ccrs.PlateCarree())

        im = ax.pcolormesh(lon, lat, data, cmap=cmap, vmin=vmin, vmax=vmax,
                           shading='auto', transform=ccrs.PlateCarree())

        ax.set_title(f'Month {month:02d}', fontsize=10)
        ax.coastlines(resolution='10m', linewidth=0.6)
        ax.add_feature(cfeature.BORDERS, linewidth=0.4)
        ax.add_feature(cfeature.LAND, edgecolor='black', facecolor='none')

        # Gridlines (bottom and left only)
        gl = ax.gridlines(draw_labels=True, linewidth=0)
        gl.top_labels = False
        gl.right_labels = False
        gl.bottom_labels = row == 2
        gl.left_labels = col == 0

    # Add a single colorbar for all subplots
    cbar_ax = fig.add_axes([0.92, 0.25, 0.015, 0.5])
    cbar = fig.colorbar(im, cax=cbar_ax)
    cbar.set_label('µg/m³')

    # Save figure
    plt.tight_layout(rect=[0, 0, 0.9, 0.95])
    filename = f"mean_concentration_no2_year_{year}.png"
    plt.savefig(os.path.join(output_folder, filename), dpi=150)
    plt.close(fig)

print("✅ Yearly temperature grids generated (3x4 months per year)!")


C:\Users\sofia\AppData\Local\Temp\ipykernel_11048\1736936664.py:72: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])
C:\Users\sofia\AppData\Local\Temp\ipykernel_11048\1736936664.py:72: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])
C:\Users\sofia\AppData\Local\Temp\ipykernel_11048\1736936664.py:72: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])
C:\Users\sofia\AppData\Local\Temp\ipykernel_11048\1736936664.py:72: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])
C:\Users\sofia\AppData\Local\Temp\ipykernel_11048\1736936664.py:72: UserWarning: This figure includes Ax

✅ Yearly temperature grids generated (3x4 months per year)!
